In [172]:
from xml.etree import ElementTree as ET
import os
import json
import re

In [173]:
def getText(a):
    text = ""
    if a.text and a.tag != "{http://www.tei-c.org/ns/1.0}note":
        text += a.text
    for child in a.getchildren():
        text += getText(child)
    return text

In [174]:
def getData(path):
    with open(path, 'r', encoding='utf-8') as file:
        str_xml = file.read()
        str_xml = re.sub('<caesura/>', ' ', str_xml)
        str_xml = re.sub('<sb[^>]*>', ' ', str_xml)
        str_xml = re.sub('</sb[^>]*>', ' ', str_xml)
        str_xml = re.sub('<rhyme[^>]*>', ' ', str_xml)
        str_xml = re.sub('</rhyme[^>]*>', ' ', str_xml)
        str_xml = re.sub('</seg>', ' </seg>', str_xml)
        root = ET.XML(str_xml)
        lines = list(root.iter('{http://www.tei-c.org/ns/1.0}l'))
        meters = [a.get('met') for a in lines]
        reals = [re.sub('|*', '', a.get('real')) for a in lines]
        text = [re.sub('\n', ' ', getText(a)).strip() for a in lines]
        return list(zip(text, meters, reals))

In [175]:
result = []

for rt, dirs, files in os.walk('data/4B4V_poems'):
    for fl in files:
        path = os.path.join(rt, fl)
        result += getData(path)

result = [a for index, a in enumerate(result) if a[0] != '']
print(result)

error: nothing to repeat at position 1

In [166]:
with open(os.path.join('data/4B4V', '4B4V.json'), 'w') as f:
        f.write(json.dumps(result))

In [169]:
import csv
f_csv = open('4B4V.csv', 'w', encoding='utf-8', newline='')
wr = csv.writer(f_csv)
wr.writerow(['sentence', 'met', 'real'])
for row in result:
    wr.writerow(row)
f_csv.close()